In [ ]:
import pandas as pd 
import numpy as np
import os

# comparing combined dataset with separate datasets

In [10]:
import pandas as pd
import numpy as np

def pred_dataset(file_names):
    source_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/CO2_adsorption/new_data'
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    for file_name in file_names:
        temp_data = pd.read_excel(os.path.join(source_path,file_name+'-02-01-2022.xlsx'),skiprows= 1 )
        
        temp_data = temp_data.dropna(axis=0,how = 'any',subset = ["BET",'Vt','Vmic','Vmeso'])
        temp_data = temp_data[temp_data['Pressure']>0.01]
        #temp_data = temp_data[temp_data['Vmic']<2]
        index = list(set(temp_data['Index'].values))
        print(len(index))
        test_index= np.random.choice(index,int(0.2*len(index)),replace=False)
        train_x = temp_data.loc[~temp_data['Index'].isin( test_index)]
        test_x = temp_data.loc[temp_data['Index'].isin(test_index)]
        
        train_df = pd.concat([train_df,train_x],axis=0)
        test_df = pd.concat([test_df,test_x],axis =0)
    return train_df,test_df

In [11]:
train_df,test_df = pred_dataset(['CO2','Methane','Ethane&Ethylene','CFCs'])

1535
676
276
31


# select some groups from the CO2 big dataset
def small_dataset():
    source_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/CO2_adsorption/new_data'
   
    temp_data = pd.read_excel(os.path.join(source_path,'CO2-01-10-2022.xlsx'),skiprows= 1 )
        
    temp_data = temp_data.dropna(axis=0,how = 'any',subset = ["BET",'Vt',])
    temp_data = temp_data[temp_data['Pressure']>0.01]
    #temp_data = temp_data[temp_data['Vmic']<2]
    index = list(set(temp_data['Index'].values))
    print(len(index))
    test_index= np.random.choice(index,394,replace=False)
    test_x = temp_data.loc[temp_data['Index'].isin(test_index)]
    test_x.to_excel(os.path.join(source_path,'CO2_small-01-10-2022.xlsx'),startrow =1)
    

# List of optmizable parameters for different models

In [3]:
#SVM
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor,\
    BaggingRegressor,ExtraTreesRegressor,RandomForestRegressor
from lightgbm import LGBMRegressor  
  
n_estimators = [50,100,120,150,180,200]

# define different models#('SVR',SVR(max_iter=10000)),
models = [
    #('DT',DecisionTreeRegressor(random_state=42)),\
     #('ADBR',AdaBoostRegressor(random_state=42)), 
    #("GBR",GradientBoostingRegressor(random_state=42)),\
    #('BG',BaggingRegressor(random_state=42,n_jobs=-1)),
    #('ETR',ExtraTreesRegressor(random_state=42,n_jobs=-1)),\
    #('RF',RandomForestRegressor(n_jobs=-1,random_state=42)),
   # ('LGBM',LGBMRegressor(n_jobs = -1,random_state = 42)),\
    #('BGLGBM',BaggingRegressor(LGBMRegressor(n_estimators = 200, n_jobs = -1,random_state = 42), random_state=42,n_jobs=-1)),\
    ('BGETR',BaggingRegressor(ExtraTreesRegressor(n_estimators = 180,random_state=42,n_jobs=6),random_state=42,n_jobs=-1))
    ]

# set search parameters grid for different models
para_grids = { #'SVR':{'kernel':['linear','poly','rbf','sigmoid','precomputed']},\
    'DT':{'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson']},\
    'ADBR':{'n_estimators':n_estimators,'learning_rate':[0.1,0.5,1,2],'loss':['linear','square','exponential']},\
    'GBR':{'n_estimators':n_estimators,'learning_rate':[0.1,0.5,1,2]},\
    'BG':{'n_estimators':[10,50,100]},\
    'ETR':{'n_estimators':n_estimators},\
    'RF':{'n_estimators':n_estimators},\
    'LGBM':{'num_leaves':[10,20,30,50],'learning_rate': [0.05,0.1,0.5,1],
    'n_estimators':n_estimators},\
    'BGLGBM':{'n_estimators':[10,30,50]},\
    'BGETR':{'n_estimators':[10]}
    }

In [4]:
from sklearn.model_selection import GridSearchCV,cross_validate,GroupKFold
from sklearn.ensemble import ExtraTreesRegressor
from  sklearn.metrics import mean_squared_error,r2_score
from sklearn.utils import shuffle

def model_CV(train_x,train_y,groups,model,para_grid):

    out_cv = GroupKFold(n_splits = 5)
    result = GridSearchCV(model,para_grid,cv= out_cv.get_n_splits(groups =groups),
    scoring='neg_mean_squared_error', return_train_score=True,n_jobs=-1)
    result.fit(train_x,train_y)
    
    model_refit =model.set_params(**result.best_params_)
    train_cv = cross_validate(model_refit,train_x,train_y,groups = groups,cv =out_cv,scoring = ('r2', 'neg_mean_squared_error'))
    train_mse_cv = -train_cv['test_neg_mean_squared_error'].mean()
    train_r2_cv = train_cv['test_r2'].mean()
    
    return [train_r2_cv,train_mse_cv],result.best_params_

# model evaluation
def model_eval(model,test_x,test_y):
      
    test_pre = model.predict(test_x)
    test_r2 = r2_score(test_pre,test_y)
    test_mse = mean_squared_error(test_y,test_pre)
    return test_r2,test_mse

# comparing different models
def model_comparison(model_list,para_grids,feature_list):
    gas_list = ['total','CO2','CFCs','Methane','E&E'] #'total',
    input_feature = feature_list
    output = ['Adsorp(mmol/g)']
    result_total = []

    for gas in gas_list:
        
        if gas =='total':

            train_df_com = train_df
            test_df_com = test_df
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            groups = train_df_com['Index'].values
            train_x, train_y, groups = shuffle(train_x, train_y, groups, random_state=42)
            
            for model_name, model in model_list:

                
                result, best_param = model_CV(train_x,train_y.squeeze(),groups,model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2_total,test_mse_total = model_eval(model_refit,test_x,test_y.squeeze()) 
                for gases in gas_list[1:]:
                    test_df_com = test_df[test_df['Label']==gases]
                    test_xs = test_df_com[input_feature]
                    test_ys = test_df_com[output].values
                    test_r2,test_mse = model_eval(model_refit,test_xs,test_ys.squeeze()) 
                    result_total.append([gases,model_name+'_total',result[0],result[1],test_r2_total,test_mse_total,test_r2,test_mse,best_param])

                    print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))

            
        else:
            
            train_df_com = train_df[train_df['Label']==gas]
            test_df_com = test_df[test_df['Label']==gas]
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            groups = train_df_com['Index']
            train_x, train_y, groups = shuffle(train_x, train_y, groups, random_state=42)
           
            for model_name, model in model_list:

                result, best_param = model_CV(train_x,train_y.squeeze(),groups,model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2,test_mse = model_eval(model_refit,test_x,test_y.squeeze()) 
                result_total.append([gas,model_name+'_separate',result[0],result[1],-1,-1, test_r2,test_mse,best_param])
                
    return result_total

In [ ]:
feature_1 = ['V','S','L','BET','Vt','Temp(K)','Pressure']
feature_2 = ['V','S','L','BET','Vt',"Vmeso",'Temp(K)','Pressure']
feature_3 = ['V','S','L','BET','Vt',"Vmic",'Temp(K)','Pressure']
feature_4 = ['V','S','L','BET','Vt',"Vmic",'Vmeso','Temp(K)','Pressure']
feature_5 = ['V','S','L','BET',"Vmic",'Vmeso','Temp(K)','Pressure']
feature_list = [feature_1,feature_2,feature_3,feature_4,feature_5]
columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param']
file_name = ['Total',"Meso","Micro",'All','Vmic_meso']
for i in range(10):
    train_df,test_df = pred_dataset(['CO2','Methane','Ethane&Ethylene','CFCs'])
    #train_df.to_csv(os.path.join('./Splitted_data/',str(i)+'_train_df.csv'))
    #test_df.to_csv(os.path.join('./Splitted_data/',str(i)+'_test_df.csv'))
    for j in range(5):
        results = model_comparison(models,para_grids, feature_list[j])
        files_name = 'BG_ETR_Full_Four_gases_with_pred_Vmic_'+file_name[j]+'_result_'+str(i)+'.csv'
        pd.DataFrame(results,columns = columns).to_csv(os.path.join('./',files_name))  
        #pd.DataFrame(results,columns = ['Gas','Algo','Train_erro','Test_error']).to_csv(os.path.join('./',files_name))   

# Testing optmization with smaller sample size for carbon dioxide (CO2)

In [71]:
#SVM
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor,\
    BaggingRegressor,ExtraTreesRegressor,RandomForestRegressor
from lightgbm import LGBMRegressor  
  
n_estimators = [50,100,150,200,250,300]

# define different models#('SVR',SVR(max_iter=10000)),
models = [#('DT',DecisionTreeRegressor(random_state=42)),\
     #('ADBR',AdaBoostRegressor(random_state=42)), ("GBR",GradientBoostingRegressor(random_state=42)),\
        ('BG',BaggingRegressor(random_state=42,n_jobs=-1)), ('ETR',ExtraTreesRegressor(random_state=42,n_jobs=-1)),\
        ('RF',RandomForestRegressor(n_jobs=-1,random_state=42)), ('LGBM',LGBMRegressor(n_jobs = -1,random_state = 42)),\
            ('BGLGBM',BaggingRegressor(LGBMRegressor(n_estimators=300, n_jobs = -1,random_state = 42), random_state=42,n_jobs=-1))]

# set search parameters grid for different models
para_grids = { #'SVR':{'kernel':['linear','poly','rbf','sigmoid','precomputed']},\
   'DT':{'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson']},\
    'ADBR':{'n_estimators':n_estimators,'learning_rate':[0.1,0.5,1,2],'loss':['linear','square','exponential']},\
    'GBR':{'n_estimators':n_estimators,'learning_rate':[0.1,0.5,1,2]},\
    'BG':{'n_estimators':[10,50,100]},\
    'ETR':{'n_estimators':n_estimators},\
    'RF':{'n_estimators':n_estimators},\
    'LGBM':{'num_leaves':[10,20,30,40,50,60],'learning_rate': [0.01,0.05,0.1,0.5,1],
    'n_estimators':n_estimators},\
    'BGLGBM':{'n_estimators':[10,50,100]}
    }

In [ ]:
# read data and reduce the dataset size
def small(file_name,selected_group):
    temp = pd.read_csv(file_name)
    CO2_data = temp[temp['Label']=='CO2']
    Other_data = temp[temp['Label']!='CO2']
    CO2_index = list(set(CO2_data['Index'].values))
    selec_index = np.random.choice(CO2_index,selected_group,replace = False)
    CO2_select = CO2_data[CO2_data['Index'].isin(selec_index)]
    return pd.concat([Other_data,CO2_select],axis = 0)
# the same dataset except smaller CO2
feature_1 = ['V','S','L','BET','Vt','Temp(K)','Pressure']
feature_2 = ['V','S','L','BET',"Vmeso",'Vt','Temp(K)','Pressure']
feature_3 = ['V','S','L','BET',"Vmic",'Vt','Temp(K)','Pressure']
feature_4 = ['V','S','L','BET',"Vmic",'Vt','Vmeso','Temp(K)','Pressure']
feature_list = [feature_1,feature_2,feature_3,feature_4]
columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param']
file_name = ['Total',"Meso","Micro",'All']
for i in range(5):

    train_df  = small(os.path.join('./Splitted_data/',str(i)+'_train_df.csv'),316)
    test_df = pd.read_csv(os.path.join('./Splitted_data/',str(i)+'_test_df.csv'))
    #train_df,test_df = pred_dataset(['CO2','Methane','Ethane&Ethylene','CFCs'])
    train_df.to_csv(os.path.join('./Splitted_data/',str(i)+'_small_train_df.csv'))
    #test_df.to_csv(os.path.join('./Splitted_data/',str(i)+'_test_df.csv'))
    for j in range(1):
        results = model_comparison(models,para_grids, feature_list[j])
        files_name = 'Small_CO2_'+file_name[j]+'_result_'+str(i)+'.csv'
        pd.DataFrame(results,columns = columns).to_csv(os.path.join('./',files_name))  
        #pd.DataFrame(results,columns = ['Gas','Algo','Train_erro','Test_error']).to_csv(os.path.join('./',files_name))  

# Using pipeline with scaling

In [15]:
# Pipeline version
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor,\
    BaggingRegressor,ExtraTreesRegressor,RandomForestRegressor
from lightgbm import LGBMRegressor  
  
n_estimators = [50,100,120,150,180,200]

# define different models#('SVR',SVR(max_iter=10000)),
models = [('DT',DecisionTreeRegressor(random_state=42)),
     ('ADBR',AdaBoostRegressor(random_state=42)), ("GBR",GradientBoostingRegressor(random_state=42)),\
        ('BG',BaggingRegressor(random_state=42,n_jobs=-1)), ('ETR',ExtraTreesRegressor(random_state=42,n_jobs=-1)),\
        ('RF',RandomForestRegressor(n_jobs=-1,random_state=42)), ('LGBM',LGBMRegressor(n_jobs = -1,random_state = 42))]

# set search parameters grid for different models
para_grids = { #'SVR':{'kernel':['linear','poly','rbf','sigmoid','precomputed']},\
   'DT':{'DT__criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson']},\
    'ADBR':{'ADBR__n_estimators':n_estimators,'ADBR__learning_rate':[0.1,0.5,1,2],'ADBR__loss':['linear','square','exponential']},\
    'GBR':{'GBR__n_estimators':n_estimators,'GBR__learning_rate':[0.1,0.5,1,2]},\
    'BG':{'BG__n_estimators':[10,50,100]},\
    'ETR':{'ETR__n_estimators':n_estimators},\
    'RF':{'RF__n_estimators':n_estimators},\
    'LGBM':{'LGBM__num_leaves':[10,20,30,40,50,60],'LGBM__learning_rate': [0.01,0.05,0.1,0.5,1],
    'LGBM__n_estimators':n_estimators}
    }

In [16]:
#Pipeline version
from sklearn.model_selection import GridSearchCV,cross_validate,GroupKFold
from sklearn.ensemble import ExtraTreesRegressor
from  sklearn.metrics import mean_squared_error,r2_score
from sklearn.utils import shuffle
#new codes
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def para_convert(results,model_name):
    best = results.best_params_
    keys = list(results.best_params_.keys())
    new_keys = {key[len(model_name)+2:]:best[key] for key in keys}
    return new_keys


def model_CV(train_x,train_y,groups,model,model_name,para_grid):
    #new codes
    scaler = StandardScaler()
    pipe = Pipeline(steps = [('scaler',scaler),(model_name,model)])
    out_cv = GroupKFold(n_splits = 5)
    result = GridSearchCV(pipe,para_grid,cv= out_cv.get_n_splits(groups =groups),
    scoring='neg_mean_squared_error', return_train_score=True,n_jobs=-1)
    result.fit(train_x,train_y)

    best_params = para_convert(result,model_name)
    
    #model_refit =model.set_params(**result.best_params_)
    model_refit = model.set_params(**best_params)
    train_cv = cross_validate(model_refit,train_x,train_y,groups = groups,cv =out_cv,scoring = ('r2', 'neg_mean_squared_error'))
    train_mse_cv = -train_cv['test_neg_mean_squared_error'].mean()
    train_r2_cv = train_cv['test_r2'].mean()
    
    return [train_r2_cv,train_mse_cv], best_params

# model evaluation
def model_eval(model,test_x,test_y):
      
    test_pre = model.predict(test_x)
    test_r2 = r2_score(test_pre,test_y)
    test_mse = mean_squared_error(test_y,test_pre)
    return test_r2,test_mse

# comparing different models
def model_comparison(model_list,para_grids,feature_list):
    gas_list = ['total','CO2','CFCs','Methane','E&E']
    input_feature = feature_list
    output = ['Adsorp(mmol/g)']
    result_total = []

    for gas in gas_list:
        
        if gas =='total':

            train_df_com = train_df
            test_df_com = test_df
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            groups = train_df_com['Index'].values
            train_x, train_y, groups = shuffle(train_x, train_y, groups, random_state=42)
            
            for model_name, model in model_list:

                result, best_param = model_CV(train_x,train_y.squeeze(),groups,model,model_name,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2_total,test_mse_total = model_eval(model_refit,test_x,test_y.squeeze()) 
                for gases in gas_list[1:]:

                    test_df_com = test_df[test_df['Label']==gases]
                    test_xs = test_df_com[input_feature]
                    test_ys = test_df_com[output].values
                    test_r2,test_mse = model_eval(model_refit,test_xs,test_ys.squeeze()) 
                    result_total.append([gases,model_name+'_total',result[0],result[1],test_r2_total,test_mse_total,test_r2,test_mse,best_param])

                    print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_total',test_r2,test_mse))

            
        else:
            
            train_df_com = train_df[train_df['Label']==gas]
            test_df_com = test_df[test_df['Label']==gas]
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            groups = train_df_com['Index']
            train_x, train_y, groups = shuffle(train_x, train_y, groups, random_state=42)
           
            for model_name, model in model_list:

                result, best_param = model_CV(train_x,train_y.squeeze(),groups,model,model_name,para_grids[model_name])
                
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2,test_mse = model_eval(model_refit,test_x,test_y.squeeze()) 
                result_total.append([gas,model_name+'_separate',result[0],result[1],-1,-1, test_r2,test_mse,best_param])
                
    return result_total

In [17]:
feature_1 = ['V','S','L','BET','Vt','Temp(K)','Pressure']
feature_2 = ['V','S','L','BET',"Vmeso",'Vt','Temp(K)','Pressure']
feature_3 = ['V','S','L','BET',"Vmic",'Vt','Temp(K)','Pressure']
feature_4 = ['V','S','L','BET',"Vmic",'Vt','Vmeso','Temp(K)','Pressure']
feature_list = [feature_1,feature_2,feature_3,feature_4]
columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param']
file_name = ['Total',"Meso","Micro",'All'] 
for i in range(10):
    train_df,test_df = pred_dataset(['CO2','Methane','Ethane&Ethylene','CFCs'])
    for j in range(4):
        results = model_comparison(models,para_grids, feature_list[j])
        files_name = 'With_scaling_Four gases_'+file_name[j]+'_result_'+str(i)+'.csv'
        pd.DataFrame(results,columns = columns).to_csv(os.path.join('./',files_name))  
        #pd.DataFrame(results,columns = ['Gas','Algo','Train_erro','Test_error']).to_csv(os.path.join('./',files_name))   


1535
676
276
31
Dataset total, Algorithm DT_total, Test_r2 0.8735793799313649, Test_error 1.2530892340955515
Dataset total, Algorithm DT_total, Test_r2 0.7315368017371896, Test_error 2.6555294364733895
Dataset total, Algorithm DT_total, Test_r2 0.8384345474573228, Test_error 1.6750248602104658
Dataset total, Algorithm DT_total, Test_r2 0.8250677577186132, Test_error 1.2729875579693222
Dataset total, Algorithm ADBR_total, Test_r2 0.44240391120032085, Test_error 3.094082533690495
Dataset total, Algorithm ADBR_total, Test_r2 -0.058839078948979884, Test_error 4.595329248873996
Dataset total, Algorithm ADBR_total, Test_r2 0.16280779439460968, Test_error 6.406808481220521
Dataset total, Algorithm ADBR_total, Test_r2 0.01571660608578318, Test_error 3.132020678980291
Dataset total, Algorithm GBR_total, Test_r2 0.9152535726972966, Test_error 0.8585778585913404
Dataset total, Algorithm GBR_total, Test_r2 0.930316597271265, Test_error 0.7769138989629911
Dataset total, Algorithm GBR_total, Test_r2

In [68]:
pd.DataFrame(results,columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_mse_separa_r2','test_mse_separa_model'])

,Gas,Model_name,CV_r2,CV_mse,test_r2_total_model,test_mse_by_total_model,test_mse_separa_r2,test_mse_separa_model
0,CO2,RF,0.916850,1.134517,0.92742,1.029823,0.941160,1.103142
1,CFCs,RF,0.916850,1.134517,0.92742,1.029823,0.688753,1.993656
2,Methane,RF,0.916850,1.134517,0.92742,1.029823,0.899515,0.919708
3,E&E,RF,0.916850,1.134517,0.92742,1.029823,0.851791,0.784912
4,CO2,RF_separate,0.921323,1.171232,-1.00000,-1.000000,0.940396,1.118080
5,CFCs,RF_separate,0.817262,1.880100,-1.00000,-1.000000,0.714499,1.805199
6,Methane,RF_separate,0.912285,1.060407,-1.00000,-1.000000,0.896908,0.949832
7,E&E,RF_separate,0.887397,0.882538,-1.00000,-1.000000,0.839518,0.870219


In [57]:
temp1 = train_df.iloc[0:10,:]
temp2 = train_df.iloc[20:30,:]
temp1.shape,temp2.shape

((10, 26), (10, 26))

In [67]:
temp3 = pd.concat([temp1.set_index(temp2.index),temp2],axis =1)
temp3.shape

(10, 52)

In [ ]:
input_feature = ['S','V','L','BET',"Vmeso",'Vt','Vmic','Temp(K)','Pressure'] # break Vt
output = ['Adsorp(mmol/g)']
train_x= train_df[input_feature]
test_x = test_df[input_feature]
train_y = train_df[output].values
test_y = test_df[output].values


In [ ]:
from sklearn.preprocessing import StandardScaler
x_scaler = StandardScaler()
train_xs = x_scaler.fit_transform(train_x)
test_xs = x_scaler.transform(test_x)
for model_name, model in model_list:

    model.fit(train_xs,train_y.squeeze())
    test_pre = model.predict(test_xs)
    train_error = mean_squared_error(train_y,model.predict(train_xs))
    test_error = mean_squared_error(test_y,test_pre)
    print('Algorithm {}, Train_error {}, Test_error {}'.format(model_name,train_error,test_error))
    #plt.scatter(test_y,test_pre)

In [ ]:
from sklearn.model_selection import GridSearchCV,cross_validate,GroupKFold
from sklearn.ensemble import ExtraTreesRegressor
from  sklearn.metrics import mean_squared_error

out_cv = GroupKFold(n_splits = 5)
model = ExtraTreesRegressor(n_jobs=-1)
#model = RandomForestRegressor(n_jobs=-1)
groups = train_df['Index']
#estimator = cross_validate(model,train_x,train_y,groups = groups,cv =out_cv)
param = {'n_estimators':[50,100,200,300,400,450,500],}
result = GridSearchCV(model,param,cv= out_cv.get_n_splits(groups =groups),
scoring='neg_mean_squared_error', return_train_score=True,n_jobs=-1)
result.fit(train_x,train_y)

In [ ]:
result.predict(test_x)
#result.cv_results_

In [ ]:
model = ExtraTreesRegressor(500,n_jobs=-1,random_state=42)

#model = RandomForestRegressor(500,random_state=42)
model.fit(train_x,train_y)
test_pre = model.predict(test_x)
train_error = mean_squared_error(train_y,model.predict(train_x))
test_error = mean_squared_error(test_y,test_pre)
print('Algorithm {}, Train_error {}, Test_error {}'.format('ETR',train_error,test_error))
cross_validate(model,train_x,train_y,groups = groups,cv =out_cv,scoring = ('r2', 'neg_mean_squared_error'))

In [ ]:
for model_name, model in model_list:

    model.fit(train_x,train_y)
    test_pre = model.predict(test_x)
    train_error = mean_squared_error(train_y,model.predict(train_x))
    test_error = mean_squared_error(test_y,test_pre)
    print('Algorithm {}, Train_error {}, Test_error {}'.format(model_name,train_error,test_error))
    #plt.scatter(test_y,test_pre)

In [ ]:
def model_comparison(model_list,feature_list):
    gas_list = ['total','CO2','CFCs','Methane','E&E']
    result = []
    for gas in gas_list:
        
        # break Vt
        output = ['Adsorp(mmol/g)']
        
        if gas =='total':
            train_df_com = train_df
            test_df_com = test_df
            #input_feature = ['E','V','L','BET',"Vmeso",'Vt','Temp(K)','Pressure'] 
            input_feature = feature_list
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output]
            test_y = test_df_com[output]
            for model_name, model in model_list:

                model.fit(train_x,train_y)
                test_pre = model.predict(test_x)
                train_error = mean_squared_error(train_y,model.predict(train_x))
                test_error = mean_squared_error(test_y,test_pre)
                result.append([gas,model_name,train_error,test_error])
                for gases in gas_list[1:]:
                    
                    test_df_com = test_df[test_df['Label']==gases]
                    test_x = test_df_com[input_feature]
                    test_y = test_df_com[output]
                    test_pre = model.predict(test_x)
                    train_error = mean_squared_error(train_y,model.predict(train_x))
                    test_error = mean_squared_error(test_y,test_pre)
                    result.append([gases,model_name,train_error,test_error])
        
                print('Dataset {}, Algorithm {}, Train_error {}, Test_error {}'.format(gas,model_name,train_error,test_error))
            
        else:
            train_df_com = train_df[train_df['Label']==gas]
            test_df_com = test_df[test_df['Label']==gas]
            if gas in ['E&E','CFCs']:
                #input_feature = ['V','L','BET',"Vmeso",'Vt','Temp(K)','Pressure'] 
                input_feature = feature_list
            else:
                #input_feature = ['BET',"Vmeso",'Vt','Temp(K)','Pressure']
                input_feature = feature_list[-5:]
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output]
            test_y = test_df_com[output]
            for model_name, model in model_list:

                model.fit(train_x,train_y)
                test_pre = model.predict(test_x)
                train_error = mean_squared_error(train_y,model.predict(train_x))
                test_error = mean_squared_error(test_y,test_pre)
                print('Dataset {}, Algorithm {}, Train_error {}, Test_error {}'.format(gas,model_name,train_error,test_error))
                #plt.scatter(test_y,test_pre)
                result.append([gas,model_name,train_error,test_error])
    return result

In [ ]:
feature_1 = ['V','S','L','BET','Vt','Temp(K)','Pressure']
feature_2 = ['V','S','L','BET',"Vmeso",'Vt','Temp(K)','Pressure']
feature_3 = ['V','S','L','BET',"Vmic",'Vt','Temp(K)','Pressure']
feature_4 = ['V','S','L','BET',"Vmic",'Vt','Vmeso','Temp(K)','Pressure']
feature_list = [feature_1,feature_2,feature_3,feature_4]
file_name = ['Total',"Meso","Micro",'All']
for i in range(5):
    train_df,test_df = pred_dataset(['CO2','Methane','Ethane&Ethylene','CFCs'])
    for j in range(4):
        result = model_comparison(model_list,feature_list[j])
        files_name = 'No-H'+file_name[j]+'result'+str(i)+'.csv'
        pd.DataFrame(result,columns = ['Gas','Algo','Train_erro','Test_error']).to_csv(os.path.join('./',files_name))   


In [ ]:
gas_list = ['total','CO2','CFCs','Methane','E&E','Hydrogen']
result = []
for gas in gas_list:
    
    # break Vt
    output = ['Adsorp(mmol/g)']
    
    if gas =='total':
        train_df_com = train_df
        test_df_com = test_df
        input_feature = ['E','V','L','BET',"Vmeso",'Vt','Temp(K)','Pressure'] 
        
        
        train_x = train_df_com[input_feature]
        test_x = test_df_com[input_feature]
        train_y = train_df_com[output]
        test_y = test_df_com[output]
        for model_name, model in model_list:

            model.fit(train_x,train_y)
            test_pre = model.predict(test_x)
            train_error = mean_squared_error(train_y,model.predict(train_x))
            test_error = mean_squared_error(test_y,test_pre)
            result.append([gas,model_name,train_error,test_error])
            for gases in gas_list[1:]:
                
                test_df_com = test_df[test_df['Label']==gases]
                test_x = test_df_com[input_feature]
                test_y = test_df_com[output]
                test_pre = model.predict(test_x)
                train_error = mean_squared_error(train_y,model.predict(train_x))
                test_error = mean_squared_error(test_y,test_pre)
                result.append([gases,model_name,train_error,test_error])
    
            print('Dataset {}, Algorithm {}, Train_error {}, Test_error {}'.format(gas,model_name,train_error,test_error))
           
    else:
        train_df_com = train_df[train_df['Label']==gas]
        test_df_com = test_df[test_df['Label']==gas]
        if gas in ['E&E','CFCs']:
            input_feature = ['V','L','BET',"Vmeso",'Vt','Temp(K)','Pressure'] 
        else:
            input_feature = ['BET',"Vmeso",'Vt','Temp(K)','Pressure']
        train_x = train_df_com[input_feature]
        test_x = test_df_com[input_feature]
        train_y = train_df_com[output]
        test_y = test_df_com[output]
        for model_name, model in model_list:

            model.fit(train_x,train_y)
            test_pre = model.predict(test_x)
            train_error = mean_squared_error(train_y,model.predict(train_x))
            test_error = mean_squared_error(test_y,test_pre)
            print('Dataset {}, Algorithm {}, Train_error {}, Test_error {}'.format(gas,model_name,train_error,test_error))
            #plt.scatter(test_y,test_pre)
            result.append([gas,model_name,train_error,test_error])



In [ ]:
pd.DataFrame(result,columns = ['Gas','Algo','Train_erro','Test_error']).to_csv('./result3.csv')

# Leave-one-group-Out cross validation

In [ ]:
import numpy as np
from sklearn.model_selection import LeavePGroupsOut

In [ ]:
lpgo = LeavePGroupsOut(n_groups=2)
lpgo.get_n_splits(groups=groups)

In [ ]:
input_feature = ['E','V','L','BET',"Vmeso",'Vt','Temp(K)','Pressure']
input_feature[-5:]

In [ ]:
pd.DataFrame(result,columns = ['Gas','Algo','Train_erro','Test_error']).to_csv('./resultnew.csv')

In [ ]:
import matplotlib.pyplot as plt
#model = RandomForestRegressor(n_estimators=30,n_jobs=-1)
#model = BaggingRegressor(base_estimator=RandomForestRegressor(n_estimators=30),n_estimators=30)
model = LGBMRegressor(objective = 'regression',num_leaves = 30,learning_rate =0.5,n_estimators=60)
model.fit(train_x,train_y)
print(r2_score(test_y,model.predict(test_x))), print(mean_squared_error(test_y,model.predict(test_x),squared=True))
plt.subplot(1,2,1)
plt.scatter(test_y,model.predict(test_x))
plt.subplot(1,2,2)
plt.scatter(train_y,model.predict(train_x))

In [ ]:
y = test_y.values.squeeze()
y_pred = model.predict(test_x)
P = test_x['logP'].values

In [ ]:
from sklearn.model_selection import GridSearchCV
estimator = LGBMRegressor()
param_grid = {'num_leaves':[10,20,30,40,50,60],'learning_rate': [0.001,0.01,0.05,0.1,0.5,1],'n_estimators':[20,30,40,50,60,80,100]}
lgbm = GridSearchCV(estimator,param_grid)
lgbm.fit(train_x,train_y)


In [ ]:
lgbm.best_params_

# Pytorch NN model

In [ ]:
train_df,test_df = pred_dataset(['CO2'])

In [ ]:
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
import torch
x_scaler = StandardScaler()
y_scaler = StandardScaler()

train_x = x_scaler.fit_transform(train_df[input_feature])
test_x = x_scaler.transform(test_df[input_feature])
#without y-scaling

train_y = train_df[output].values.reshape(-1,1)


# with y_scaling
#train_y = y_scaler.fit_transform(train_df[output]).reshape(-1,1)
test_y = test_df[output].values.reshape(-1,1)
#print(train_y)

train_x = torch.Tensor(train_x).float()
train_y = torch.Tensor(train_y).float()
test_x = torch.Tensor(test_x).float()
test_y = torch.Tensor(test_y).float()

from torch.utils.data import Dataset
class CustomImageDataset(Dataset):
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs
        
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self,idx):   
        return self.inputs[idx], self.outputs[idx]

        

In [ ]:
train_dataset = CustomImageDataset(train_x,train_y.reshape(-1,1))
train_loader = DataLoader(train_dataset)
val_dataset = CustomImageDataset(test_x,test_y.reshape(-1,1))
val_loader = DataLoader(val_dataset,batch_size=500)

In [ ]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self,input_length):
        super().__init__()
        self.input_length = input_length
        self.fc1 = torch.nn.Linear(self.input_length,158)
        self.ac1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(158,158)
        self.drop1 = torch.nn.Dropout(0.14)
        self.ac2 = torch.nn.ReLU()
        self.fc3 = torch.nn.Linear(158,158)
        self.ac3 = torch.nn.ReLU()
        self.out = torch.nn.Linear(158,1)
    def forward(self,x):
        out = self.drop1(self.ac1(self.fc1(x)))
        out = self.ac2(self.fc2(out))
        out = self.ac3(self.fc3(out))
        return self.out(out)

In [ ]:
# change the weight initiation
import torch.nn as nn
def mol_init(model):
    for m in model.modules():
        if isinstance(m,nn.Linear):
            nn.init.xavier_normal_(m.weight)


In [ ]:
import torch.optim as opt
input_length = len(input_feature)
model = Net(input_length)
mol_init(model)
loss_fn = torch.nn.MSELoss()
optimizer = opt.Adam(model.parameters(),lr=0.024)
hist_loss = []
epoches = 200
for epoch in range(1,epoches+1):
    
    losses = []
    for x,y in train_loader:
        y_pred = model(x)
        loss = loss_fn(y,y_pred)
        losses.append(loss)
    lossn = sum(losses)/len(losses)
    hist_loss.append(lossn.item())
    optimizer.zero_grad()
    lossn.backward()
    optimizer.step()

    if epoch%10 ==0:
        
        y_real= torch.Tensor(())
        y_pred = torch.Tensor(())
        with torch.no_grad():
            for val_x,val_y in val_loader:
                val_y_pre = model(val_x) 
                
                y_real=torch.cat((y_real,val_y),0)
                y_pred = torch.cat((y_pred,val_y_pre),0)
    
            #y_pred = torch.Tensor(y_scaler.inverse_transform(y_pred))
            y_pred = torch.Tensor(y_pred)
    
            val_loss = loss_fn(y_real,y_pred)
        print('Epcoh: {}, Train_loss {:.4f}, Val_loss {:.4f}'.format(epoch,hist_loss[-1],val_loss))


In [ ]:
from sklearn.metrics import mean_squared_error,r2_score


from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,\
    GradientBoostingRegressor,ExtraTreesRegressor, BaggingRegressor

    
model_list = [('RF',RandomForestRegressor(n_estimators=50)),('ADB',AdaBoostRegressor(n_estimators=50)),
('GBR',GradientBoostingRegressor(n_estimators=50)),('ETR',ExtraTreesRegressor(n_estimators=50)),
('BGR',BaggingRegressor(ExtraTreesRegressor(n_estimators=50)))]
input_feature = ['E','V','L','BET',"Vmeso",'Vt','Vmic','Temp(K)','Pressure'] # break Vt
output = ['Adsorp(mmol/g)']
train_x= train_df[input_feature].values
test_x = test_df[input_feature].values
train_y = train_df[output].values
test_y = test_df[output].values


for model_name, model in model_list:

    model.fit(train_x,train_y)
    test_pre = model.predict(test_x)
    train_error = mean_squared_error(train_y,model.predict(train_x))
    test_error = mean_squared_error(test_y,test_pre)
    print('Algorithm {}, Train_error {}, Test_error {}'.format(model_name,train_error,test_error))
    #plt.scatter(test_y,test_pre)

# Leave-one-out looping to remove poor data points

In [ ]:
import pandas as pd 
import os
import numpy as np
file_name = 'Hydrogen'
source_path = '/Users/kai/Documents/Desktop/CO2_adsorption/new_data'

temp_data = pd.read_excel(os.path.join(source_path,file_name+'-01-01-2022.xlsx'),skiprows=1)

In [ ]:
#os.listdir(source_path)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

test_mse = []
temp_data = temp_data.dropna(axis=0,how = 'any',subset = ["BET","Vt",'Vmic'])
for i in list(set(temp_data['Index'].values)):
    
    #if i not in ex_list:
    
        test_data = temp_data[temp_data['Index']==i]
        train_data = temp_data[temp_data['Index']!=i]
        
        input_feature = ['E','V','L','BET','Vt','Vmic','Temp(K)','Pressure']
        output = ['Adsorp(mmol/g)']
        train_x = train_data[input_feature]
        train_y = train_data[output].values.reshape(-1)
        test_x = test_data[input_feature]
        test_y = test_data[output].values.reshape(-1)
        model = RandomForestRegressor(n_estimators=200,n_jobs=-1)
        model.fit(train_x,train_y)
        test_mse.append([i,r2_score(test_y,model.predict(test_x)),mean_squared_error(test_y,model.predict(test_x))])
        if len(test_mse)%20==0:
            print(test_mse[-1])

In [ ]:
pd.DataFrame(test_mse,columns=['group','r2','mse']).to_csv(os.path.join(source_path,'Hydrogen_LOO2_0109_del_del_mic_del_del.csv'))

In [ ]:
ex_list = [132,173,182,191,212,711,739,
825,828,830,833,876,957,1136,1201,
1227,1457,1516,1535,1536,1675,1953,
1985,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2040,2042]

In [ ]:
import numpy as np
def square(x: np.ndarray):
    return np.power(x,2)


In [ ]:
square(np.array([1,3]))

In [ ]:
from typing import List